In [4]:
import scanpy as sc
import numpy as np


"""
so basically i was really stuggled with the size of the dataset and i needed to do something sbout it,
so here just take the subset
also the data from the paper - wasnt able to get it to server because you need to download the full
zip file and i dont have that space left on my account :((((

i think this should be more or less ready to go to encoder
"""

#load the data with backed = "r", otherwise i just got a RAM probelm

DATA_PATH = "../data/bulk_processed_genes.h5ad"
adata_meta = sc.read_h5ad(DATA_PATH, backed="r")
print("Full data shape:", adata_meta.shape)

# subset of the data - otherwise i got RAM error - again
n_cells_keep = 5000
idx = np.random.choice(adata_meta.n_obs, size=n_cells_keep, replace=False)
idx.sort()

# Load only those cells fully into memory
adata = sc.read_h5ad(DATA_PATH)
adata = adata[idx, :] 

print("Loaded subset:", adata.shape)

#Filter weakly expressed genes using existing stats - they did that in the paper
#and this is basically only preprocessing we need to do
if "n_cells_by_counts" in adata.var.columns:
    keep = adata.var["n_cells_by_counts"] >= 20
    adata = adata[:, keep]  
else:
    print("No precomputed n_cells_by_counts; skipping")

print("After filtering:", adata.shape)

sc.pp.highly_variable_genes(adata, n_top_genes=2000)

adata = adata[:, adata.var["highly_variable"]] 
print("Final shape:", adata.shape)

# new smaller dataset
adata.write_h5ad("adata_preprocessed_50k_2kgenes.h5ad")

Full data shape: (19882, 45263)


MemoryError: Unable to allocate 1.90 GiB for an array with shape (511090881,) and data type int32

In [2]:
print("Done!")

Done!


In [5]:
import scanpy as sc
import numpy as np

"""
This version avoids loading the full dataset into memory.
We first open it in backed mode to get dimensions and metadata,
then load only a random subset fully into memory for downstream processing.
"""

DATA_PATH = "../data/bulk_processed_genes.h5ad"

# Open in backed mode (lazy loading, doesn't load X into RAM)
adata_meta = sc.read_h5ad(DATA_PATH, backed="r")
print("Full data shape:", adata_meta.shape)

# Take a random subset of cells to avoid MemoryError
n_cells_keep = 5000
idx = np.random.choice(adata_meta.n_obs, size=n_cells_keep, replace=False)
idx.sort()

# Load only the selected subset fully into memory
adata = adata_meta[idx, :].to_memory()
print("Loaded subset:", adata.shape)

# Filter weakly expressed genes (if precomputed stats exist)
if "n_cells_by_counts" in adata.var.columns:
    keep = adata.var["n_cells_by_counts"] >= 20
    adata = adata[:, keep]
else:
    print("No precomputed n_cells_by_counts; skipping")

print("After filtering:", adata.shape)

# Identify highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=2000)

# Keep only the highly variable genes
adata = adata[:, adata.var["highly_variable"]]
print("Final shape:", adata.shape)

# Save the smaller dataset for downstream analysis
adata.write_h5ad("adata_preprocessed_5k_2kgenes.h5ad")


Full data shape: (19882, 45263)


MemoryError: Unable to allocate 490. MiB for an array with shape (128356315,) and data type int32

In [6]:
import scanpy as sc
import numpy as np

DATA_PATH = "../data/bulk_processed_genes.h5ad"

# 1️⃣ Open in backed mode
adata = sc.read_h5ad(DATA_PATH, backed="r")
print("Full data shape:", adata.shape)

# 2️⃣ Pick a subset of cell indices
n_cells_keep = 5000
idx = np.random.choice(adata.n_obs, size=n_cells_keep, replace=False)
idx.sort()
subset_obs_names = adata.obs_names[idx]

# 3️⃣ Create a new .h5ad with only the subset
# This avoids loading the full X into memory
adata_subset = adata[subset_obs_names, :]
adata_subset.write_h5ad("adata_subset.h5ad", chunks=True)

print("Subset saved. Shape:", adata_subset.shape)


Full data shape: (19882, 45263)


TypeError: AnnData.write_h5ad() got an unexpected keyword argument 'chunks'

In [7]:
import scanpy as sc
import numpy as np

DATA_PATH = "../data/bulk_processed_genes.h5ad"

# 1️⃣ Open in backed mode
adata = sc.read_h5ad(DATA_PATH, backed="r")
print("Full data shape:", adata.shape)

# 2️⃣ Pick a random subset of cells
n_cells_keep = 5000
idx = np.random.choice(adata.n_obs, size=n_cells_keep, replace=False)
idx.sort()
subset_obs_names = adata.obs_names[idx]

# 3️⃣ Create a subset AnnData object (view, not fully loaded yet)
adata_subset = adata[subset_obs_names, :].copy()  # .copy() loads only the subset into memory

print("Subset loaded into memory. Shape:", adata_subset.shape)

# 4️⃣ Filter weakly expressed genes
if "n_cells_by_counts" in adata_subset.var.columns:
    keep = adata_subset.var["n_cells_by_counts"] >= 20
    adata_subset = adata_subset[:, keep]
else:
    print("No precomputed n_cells_by_counts; skipping")

print("After filtering:", adata_subset.shape)

# 5️⃣ Identify highly variable genes
sc.pp.highly_variable_genes(adata_subset, n_top_genes=2000)

# 6️⃣ Keep only the highly variable genes
adata_subset = adata_subset[:, adata_subset.var["highly_variable"]]
print("Final shape:", adata_subset.shape)

# 7️⃣ Save the smaller dataset
adata_subset.write_h5ad("adata_preprocessed_5k_2kgenes.h5ad")


Full data shape: (19882, 45263)


ValueError: To copy an AnnData object in backed mode, pass a filename: `.copy(filename='myfilename.h5ad')`. To load the object into memory, use `.to_memory()`.

In [ ]:
adata = sc.read_h5ad("adata_subset.h5ad")
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata = adata[:, adata.var["highly_variable"]]
adata.write_h5ad("adata_preprocessed_5k_2kgenes.h5ad")
